# Effect of using thresholds with softmax

This notebook shows the effect of choosing thresholds for the softmax layer in classification problems.

The usual approach is to choose the maximum of the predicted probabilities:

... code here <<<<<<<<

However, this gloses over the fact that sometimes two or more classes have very close predicted probabilities.

Using the MNIST dataset, we will investigate the effect of settings thresholds to make predictions, instead of simply the maximum probability.

Sources:

- [Tensorflow MNIST example](https://www.tensorflow.org/datasets/keras_example)
- [Multi-label vs. Multi-class Classification: Sigmoid vs. Softmax](https://glassboxmedicine.com/2019/05/26/classification-sigmoid-vs-softmax/)
- [Interpreting logits: sigmoid vs softmax](https://web.stanford.edu/~nanbhas/blog/sigmoid-softmax.html)

## Load MNIST and create the training/test pipelines

The code in this section is from the [TensorFlow MNIST example](https://www.tensorflow.org/datasets/keras_example).

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds

Load MNIST.

In [6]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  2.99 file/s]
Dataset mnist downloaded and prepared to /Users/cgarbin/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.



Build training pipeline.

In [7]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

Build test pipeline.

In [8]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

## Train a model

In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

Use early stopping to get a decent model without investing too much time in training. The main purspose of this notebook is not to get high accuracy. It's to demonstrate the effect of softmax thresholds.

In [20]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=2)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
    callbacks=[callback]
)

Epoch 1/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0010 - sparse_categorical_accuracy: 0.9998 - val_loss: 0.1310 - val_sparse_categorical_accuracy: 0.9771
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 2.0192e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1264 - val_sparse_categorical_accuracy: 0.9787
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 1.1655e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1260 - val_sparse_categorical_accuracy: 0.9791
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 9.5013e-05 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1259 - val_sparse_categorical_accuracy: 0.9794
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 8.1477e-05 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1260 - val_sparse_categorical_accuracy: 0.9790
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 7.1974e-05